<a href="https://colab.research.google.com/github/Dohy-Lee/Do_it-BERT_GPT/blob/main/qa_deploy_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ratsnlp
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 677 kB/s 
     |████████████████████████████████| 2.8 MB 8.5 MB/s 
     |████████████████████████████████| 57 kB 6.4 MB/s 
     |████████████████████████████████| 582 kB 82.8 MB/s 
     |████████████████████████████████| 596 kB 76.9 MB/s 
     |████████████████████████████████| 419 kB 64.6 MB/s 
     |████████████████████████████████| 141 kB 64.7 MB/s 
     |████████████████████████████████| 3.3 MB 44.1 MB/s 
     |████████████████████████████████| 880 kB 41.4 MB/s 
     |████████████████████████████████| 101 kB 13.5 MB/s 
     |████████████████████████████████| 96 kB 6.9 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=94785a6391642f959207af20e7740977aab5bde39e4909ecec523340325ab01d
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Suc

##인퍼런스 설정

In [2]:
from ratsnlp.nlpbook.qa import QADeployArguments
args = QADeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-qa",
    max_seq_length=128,
    max_query_length=32,
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-qa/epoch=0-val_loss=0.46.ckpt


##토크나이저 및 모델 불러오기

In [3]:
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

In [6]:
import torch
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu"),
)

In [11]:
from transformers import BertConfig, BertForQuestionAnswering
pretrained_model_config=BertConfig.from_pretrained(
    args.pretrained_model_name,
)

In [15]:
model=BertForQuestionAnswering(pretrained_model_config)
model.load_state_dict({k.replace("model.",""): v for k,v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

##인퍼런스

In [16]:
def inference_fn(question, context):
  if question and context:
    truncated_query = tokenizer.encode(#질문을 토큰화하고, 인덱싱하되 max_query_length보다 길면 이에 맞게 자르기
        question,
        add_special_tokens=False,
        truncation=True,
        max_length=args.max_query_length
    )
    inputs = tokenizer.encode_plus( #앞서 처리한 질문(truncated_query)을 지문과 함께 토큰화 하고 인덱싱하되 전체 길이가 max_seq_length보다 길면 지문 자르기(truncation='only_sencd')
        text=truncated_query,
        text_pair=context,
        truncation='only_second',
        padding='max_length',
        max_length=args.max_seq_length,
        return_token_type_ides=True,
    )
    with token.no_grad():
      outputs=model(**{k: torch.tensor([v]) for k,v in inputs.items()})
      start_pred= outputs.start_logits.argmax(dim=-1).item() #정답의 시작 위치와 관련된 로짓(outputs.start_logits)에서 가장 큰 값이 가리키는 토큰 위치 찾기
      end_pred = outputs.end_logits.argmax(dim=-1).item() # 정답의 끝 위치와 관련된 로짓(outputs.end_logits)에서 가장 큰 값이 가리키는 토큰 위치 찾기
      pred_text=tokenizer.decode(inputs['input_ids'][start_pred:end_pred+1]) #정답 시작부터 끝까지의 토큰들을 이어붙여 정답 만들기
  else:
    pred_text=''
  return {
      'question':question,
      'context':context,
      'answer':pred_text
  }

##웹 서비스 구현


In [17]:
!mkdir /root/.ngrok2 && echo "authtoken: 토큰키" > /root/.ngrok2/ngrok.yml

mkdir: cannot create directory ‘/root/.ngrok2’: File exists


In [ ]:
from ratsnlp.nlpbook.qa import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.qa.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://2c16-35-231-245-153.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [12/Aug/2022 11:19:24] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Aug/2022 11:19:24] "GET /favicon.ico HTTP/1.1" 404 -
